# Setup

## 1. Docker Image 저장 경로 변경

Change Docker image path to EBS
- SageMaker 노트북 인스턴스일 경우 docker image 저장소의 공간이 작아 마운트한 EBS로 변경을 진행합니다.
SageMaker 노트북 인스턴스에서 로컬 모드 디버깅 시 종종 No space left 관련 오류가 발생합니다. 따라서, 도커 이미지/컨테이너가 저장될 폴더를 SageMaker EBS (Amazon Elastic Block Store) 볼륨으로 변경하는 것을 권장합니다. 도커 이미지/컨테이너는 기본적으로 EBS가 아닌 루트 볼륨에 저장하기 때문에(루트 볼륨의 크기는 사용자가 임의로 조정할 수 없습니다!) 고용량의 이미지들을 빌드하면 용량이 꽉 차기 때문입니다.

In [1]:
%%bash

#!/usr/bin/env bash

echo '{
    "runtimes": {
        "nvidia": {
            "path": "nvidia-container-runtime",
            "runtimeArgs": []
        }
    }
}' > daemon.json

sudo cp daemon.json /etc/docker/daemon.json && rm daemon.json

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

sudo docker info | grep Root

Add data-root and default-shm-size=10G


Redirecting to /bin/systemctl restart docker.service


Docker Restart
 Docker Root Dir: /home/ec2-user/SageMaker/.container/docker


## 2. 환경 변수 설정
- HF 에서 사용하는 디폴트 경로를 변경 합니다. 
- 아래의 sh 파일을 실행하면, 아래의 변수가 환경 변수에 저장이 되어서, 
`os.getenv('TRANSFORMERS_CACHE')` 의 명령어를 통해서 값도 불러 올 수 있습니다. 
```
export TRANSFORMERS_CACHE=/home/ec2-user/SageMaker/.cache
export HF_DATASETS_CACHE=/home/ec2-user/SageMaker/.cache
export HF_HOME=/home/ec2-user/SageMaker/.cache

```

In [2]:
! ./setup_env_vars.sh

export TRANSFORMERS_CACHE=/home/ec2-user/SageMaker/.cache is export TRANSFORMERS_CACHE=/home/ec2-user/SageMaker/.cache
Added TRANSFORMERS_CACHE, HF_DATASETS_CACHE, HF_HOME to .bashrc
TRANSFORMERS_CACHE is now set to: /home/ec2-user/SageMaker/.cache
HF_DATASETS_CACHE is now set to: /home/ec2-user/SageMaker/.cache
HF_HOME is now set to: /home/ec2-user/SageMaker/.cache


## 3. Jupyter Server 재실행
#### [중요] 아래를 실행하면 모든 노트북의 커널이 중지됩니다. 유의해서 사용 하세요.

In [ ]:
import os
os.system('sudo systemctl restart jupyter-server')